# Webscraping a čištění dat

#### Cílem tohoto projektu je získat z webových stránek AAA Auto data ohledně nabídek vozidel. K **webscrapingu** využívám library Selenium.

#### Data by měla obsahovat následující proměnné: 
- značka
- model
- cena [Kč]
- rok uvedení do provozu
- tachometr [km]
- výkon motoru [kW]
- převodovka
- barva
- karoserie
- palivo

#### Následně je nutné docílit těchto bodů v rámci **čištění dat**:
- data neobsahují duplicity
- data neobsahují chybějící položky (ty jsou buď smazány nebo nahrazeny nejčastější hodnotou)
- datové typy jsou správně
- data jsou konzistentní (uniformní kategorické proměnné, absense nepatřičných znaků)
- proměnné obsahují správné hodnoty (nejsou např. prohozené)
- data jsou připravena k aplikaci ML modelu - každý sloupec obsahuje proměnnou, každý řádek obsahuje unikátní záznam

In [511]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

## Část 1 - Webscraping

#### Inicializace driveru a url odkazu

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.134/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\uzivatel\.wdm\drivers\chromedriver\win32\103.0.5060.134]
C:\Users\uzivatel\AppData\Local\Temp\ipykernel_9356\349924444.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
page_url = "https://www.aaaauto.cz/ojete-vozy/"

driver.get(page_url)

#### Kliknutí na "přijmout vše" cookies tlačítko

In [5]:
from selenium.webdriver.common.by import By
time.sleep(3)
driver.find_element(By.CLASS_NAME, 'cookiebarBtn').click()

#### Extrakce nadpisů nabídek z první stránky

nadpis_auta = driver.find_elements(By.CLASS_NAME, 'primary.notranslate')
nadpis_auta

nadpis_auta[0].text

#### Transformace nadpisů na první straně do listu

In [9]:
titles = []
for nadpis in nadpis_auta:
    title = nadpis.text
    titles.append(title)
len(titles)

23

In [75]:
titles[:6]

['Škoda Karoq, 2021',
 'Škoda Octavia, 2021',
 'od 4 275 Kč',
 'Škoda Octavia, 2021',
 'od 3 825 Kč',
 'Škoda Karoq, 2020']

#### Extrakce odkazů nabídek z první stránky

In [11]:
odkazy = driver.find_elements(By.CLASS_NAME, 'fullSizeLink')
odkazy

[<selenium.webdriver.remote.webelement.WebElement (session="3e15c7e7333b6de777197a46564113f5", element="39d6f87f-275f-4912-a7fd-cdb99066a13e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3e15c7e7333b6de777197a46564113f5", element="ee27ae8f-830a-4c1c-baa6-4f5380741dbd")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3e15c7e7333b6de777197a46564113f5", element="cf5d1dd6-87c8-4714-90ac-761d9567da0e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3e15c7e7333b6de777197a46564113f5", element="480664c6-c56c-45b1-8e82-178c4b9ab404")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3e15c7e7333b6de777197a46564113f5", element="70ff9260-1f13-41ed-8b8f-3a762cf01c75")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3e15c7e7333b6de777197a46564113f5", element="43c7beaf-aaa4-4ac4-935f-ac19bad5785f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3e15c7e7333b6de777197a46564113f5", element="9f5effcd-8849-4e5f-ace7-2f

In [12]:
odkazy[0].get_attribute('href')

'https://www.aaaauto.cz/cz/skoda-karoq/car.html?id=502014742#'

#### Transformace odkazů na první straně do listu

In [13]:
urls = []
for odkaz in odkazy:
    url = odkaz.get_attribute('href')
    urls.append(url)
len(urls)

10

In [76]:
urls[:6]

['https://www.aaaauto.cz/cz/skoda-karoq/car.html?id=502014742#',
 'https://www.aaaauto.cz/cz/skoda-octavia/car.html?id=483692081#&promo=gm',
 'https://www.aaaauto.cz/cz/skoda-octavia/car.html?id=489214916#&promo=gm',
 'https://www.aaaauto.cz/cz/skoda-karoq/car.html?id=504615380#&promo=r',
 'https://www.aaaauto.cz/cz/vw-passat/car.html?id=504883170#&promo=r',
 'https://www.aaaauto.cz/cz/vw-golf/car.html?id=493294565#&promo=r']

#### Extrakce nadpisů a odkazů ze všech (cca 1100) stran (na každé straně je 10 nabídek vozů)

In [16]:
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


titles_list = []
urls_list = []

while True:
    try:
        next_link = WebDriverWait(driver,4).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".svg-replaced.iconMin.colored.rotate180")))
        next_link.click()
        time.sleep(4)
        
        # extrakce všech nadpisů
        elementy_nadpisy = driver.find_elements(By.CLASS_NAME, 'primary.notranslate')
        for element in elementy_nadpisy:
            nadpis = element.text
            titles_list.append(nadpis)
        
        # extrakce všech odkazů
        elementy_odkazy = driver.find_elements(By.CLASS_NAME, 'fullSizeLink')
        for element_odkaz in elementy_odkazy:
            url = element_odkaz.get_attribute('href')
            urls_list.append(url)
        
        
    except TimeoutException:
        break

In [29]:
len(urls_list)

11424

In [30]:
len(titles_list)

27187

In [31]:
titles_complete = titles + titles_list
len(titles_complete)

27210

In [32]:
urls_complete = urls + urls_list
len(urls_complete)

11434

#### Očištění listu s nadpisy o nerelevantní položky

In [34]:
smazat = "Kč"

nadpisy_nabidek = [ x for x in titles_complete if smazat not in x ]
len(nadpisy_nabidek)

16010

#### Přeskočení doporučených vozů na každé stránce (ponechání prvních 10 a přeskočení dalších 4 nabídek)

In [35]:
nadpisy_nabidek = [x for i in range(0, len(nadpisy_nabidek), 14) for x in nadpisy_nabidek[i:i+10]]
len(nadpisy_nabidek)

11438

In [36]:
nadpisy_nabidek = nadpisy_nabidek[:-4]
len(nadpisy_nabidek)

11434

#### Vytvoření unikátního ID pro každou nabídku

In [45]:
counter = 0
new_list = []
for each_item in nadpisy_nabidek:
    new_list.append(each_item + "_" + str(counter))
    counter += 1

In [51]:
len(new_list)

11434

#### Spojení listu nadpisů a listu odkazů do jedné dictionary

In [52]:
vysl = {new_list[i]: urls_complete[i] for i in range(len(new_list))}

In [54]:
len(vysl)

11434

#### Vizuální kontrola, zda nadpisy a url souhlasí

In [395]:
df = pd.DataFrame(vysl.items(), columns=['nadpis','url'])
df

,nadpis,url
0,"Škoda Karoq, 2021_0",https://www.aaaauto.cz/cz/skoda-karoq/car.html...
1,"Škoda Octavia, 2021_1",https://www.aaaauto.cz/cz/skoda-octavia/car.ht...
2,"Škoda Octavia, 2021_2",https://www.aaaauto.cz/cz/skoda-octavia/car.ht...
3,"Škoda Karoq, 2020_3",https://www.aaaauto.cz/cz/skoda-karoq/car.html...
4,"Volkswagen Passat, 2020_4",https://www.aaaauto.cz/cz/vw-passat/car.html?i...
...,...,...
11429,"Volkswagen Golf Sportsvan, 2014_11429",https://www.aaaauto.cz/cz/vw-golf-sportsvan/ca...
11430,"Škoda Octavia Scout, 2013_11430",https://www.aaaauto.cz/cz/skoda-octavia-scout/...
11431,"Mercedes-Benz B 200 CDI, 2007_11431",https://www.aaaauto.cz/cz/mercedes-b/car.html?...
11432,"Jeep Patriot, 2007_11432",https://www.aaaauto.cz/cz/jeep-patriot/car.htm...


In [56]:
new_dict = df.to_dict('records')
new_dict[:7]

[{'nadpis': 'Škoda Karoq, 2021_0',
  'url': 'https://www.aaaauto.cz/cz/skoda-karoq/car.html?id=502014742#'},
 {'nadpis': 'Škoda Octavia, 2021_1',
  'url': 'https://www.aaaauto.cz/cz/skoda-octavia/car.html?id=483692081#&promo=gm'},
 {'nadpis': 'Škoda Octavia, 2021_2',
  'url': 'https://www.aaaauto.cz/cz/skoda-octavia/car.html?id=489214916#&promo=gm'},
 {'nadpis': 'Škoda Karoq, 2020_3',
  'url': 'https://www.aaaauto.cz/cz/skoda-karoq/car.html?id=504615380#&promo=r'},
 {'nadpis': 'Volkswagen Passat, 2020_4',
  'url': 'https://www.aaaauto.cz/cz/vw-passat/car.html?id=504883170#&promo=r'},
 {'nadpis': 'Volkswagen Golf, 2020_5',
  'url': 'https://www.aaaauto.cz/cz/vw-golf/car.html?id=493294565#&promo=r'},
 {'nadpis': 'Škoda Scala, 2021_6',
  'url': 'https://www.aaaauto.cz/cz/skoda-scala/car.html?id=499163818#'}]

#### Extrakce ceny a detailních informací o každém vozidle z url každé individuální nabídky

In [5]:
info_list = []

for element in new_dict:
    # rozkliknutí url nabídky 
    driver.get(element['url'])
    
    ceny = driver.find_elements(By.XPATH, "//*[@class='infoBoxNavTitle' or @class='notranslate']")
    for cena in ceny:
        info_list.append({'auto': element['nadpis'], 'cena': cena.text})
    
    informace = driver.find_elements(By.CLASS_NAME, 'transparentTable')
    for info in informace:
        info_list.append({'auto': element['nadpis'], 'info': info.text})


#### Webscraping přerušen
#### Vzhledem k časové náročnosti webscrapingu (cca 12 hod) for loop přerušen po několika hodinách, z celkových 11438 nabízek vozů je k dispozici vzorek 2939 nabídek vozů

In [93]:
len(info_list)

62637

#### Smazání prázdných hodnot v dictionary

In [96]:
info_list_bez_prazdnych = [ele for ele in ({key: val for key, val in sub.items() if val} for sub in info_list) if ele]

In [98]:
len(info_list_bez_prazdnych)

62637

#### Tvorba dataframu

In [695]:
df_auta = pd.DataFrame(info_list_bez_prazdnych)
df_auta.head()

,auto,cena,info
0,"Škoda Karoq, 2021_0",Škoda,NaN
1,"Škoda Karoq, 2021_0",Škoda Karoq,NaN
2,"Škoda Karoq, 2021_0",Cena\n880 000 Kč,NaN
3,"Škoda Karoq, 2021_0",152 727 Kč,NaN
4,"Škoda Karoq, 2021_0",NaN,NaN


#### Počet stažených nabídek

In [696]:
df_auta['auto'].nunique()

2939

#### Export raw scraped dat do csv

In [697]:
df_auta.to_csv('auta_scraped.csv',encoding='utf-8-sig')

## Část 2 - Čištění dat

In [128]:
import re

In [835]:
df_auta = pd.read_csv('auta_scraped.csv',usecols=['auto', 'cena','info'])

In [843]:
df_auta[:12]

,auto,cena,info
0,"Škoda Karoq, 2021_0",Škoda,NaN
1,"Škoda Karoq, 2021_0",Škoda Karoq,NaN
2,"Škoda Karoq, 2021_0",Cena\n880 000 Kč,NaN
3,"Škoda Karoq, 2021_0",152 727 Kč,NaN
4,"Škoda Karoq, 2021_0",NaN,NaN
...,...,...,...
7,"Škoda Karoq, 2021_0",NaN,NaN
8,"Škoda Karoq, 2021_0",NaN,NaN
9,"Škoda Karoq, 2021_0",NaN,NaN
10,"Škoda Karoq, 2021_0",NaN,Rok uvedení do provozu 2021\nTachometr 38 188 ...


In [844]:
# text převést do sloupců: Rok, Tachometr, Motor, Převodovka, Barva, Karoserie, Palivo

print(df_auta['info'][10])
print(df_auta['info'][11])

Rok uvedení do provozu 2021
Tachometr 38 188 km
Motor 2.0 TDI, 110kW, 4x4
Převodovka Automat / 7 stupňů
Barva modrá-metalíza
Stupeň výbavy Style Plus
STK 04/2025
Karoserie SUV
Palivo Diesel
Míst k sezení 5 míst
Emisní norma EURO6
Emise CO₂ 156 g/km
Historie Elektronická servisní knížka, Koupeno nové v ČR


In [845]:
df_auta.shape

(62637, 3)

In [846]:
df_auta_beznan = df_auta.dropna(subset=['cena','info'],how='all').reset_index(drop=True)

In [847]:
df_auta_beznan.shape

(24969, 3)

In [848]:
df_auta_beznan.head(10)

,auto,cena,info
0,"Škoda Karoq, 2021_0",Škoda,NaN
1,"Škoda Karoq, 2021_0",Škoda Karoq,NaN
2,"Škoda Karoq, 2021_0",Cena\n880 000 Kč,NaN
3,"Škoda Karoq, 2021_0",152 727 Kč,NaN
4,"Škoda Karoq, 2021_0",NaN,Rok uvedení do provozu 2021\nTachometr 38 188 ...
5,"Škoda Karoq, 2021_0",NaN,Karoserie SUV\nPalivo Diesel\nMíst k sezení 5 ...
6,"Škoda Octavia, 2021_1",Škoda,NaN
7,"Škoda Octavia, 2021_1",Škoda Octavia,NaN
8,"Škoda Octavia, 2021_1",Cena\n830 000 Kč\n760 000 Kč,NaN
9,"Škoda Octavia, 2021_1",830 000 Kč\n760 000 Kč,NaN


In [849]:
df_auto_cena = df_auta_beznan[['auto','cena']]

In [850]:
df_auto_cena['cumcount'] = df_auto_cena.groupby(['auto']).cumcount()+1

In [851]:
df_auto_cena = df_auto_cena.pivot(index='auto',columns='cumcount',values='cena').reset_index()

#### Je zde 5 různých sloupců s cenami - vybrání té správné ceny u dané nabídky na základě různých podmínek

In [852]:
word = "Cena"
df_auto_cena['cont_1'] = df_auto_cena[3].str.contains(word)

In [853]:
df_auto_cena['spravna_cena'] = df_auto_cena.apply(lambda x: x[3] if x['cont_1']==True else x[4], axis=1)

In [854]:
df_auto_cena = df_auto_cena[['auto',1,2,'spravna_cena']].rename(columns = {'auto':'id', 1:'znacka',2:'model','spravna_cena':'cena'})

In [855]:
ceny = df_auto_cena['cena'].str.split(pat="\n", expand=True)

In [856]:
slovo1="Cena"
cena = ceny.apply(lambda x: x[0] if x[0] != slovo1 else (x[2] if x[2] != None else x[1]), axis=1)

In [857]:
df_auto_cena['cena'] = cena

In [858]:
df_auto_cena.head()

cumcount,id,znacka,model,cena
0,"Alfa Romeo 159, 2007_371",Alfa Romeo,Alfa Romeo 159,74 000 Kč
1,"Alfa Romeo 159, 2008_1894",Alfa Romeo,Alfa Romeo 159,106 000 Kč
2,"Alfa Romeo Giulietta, 2018_2882",Alfa Romeo,Alfa Romeo Giulietta,311 000 Kč
3,"Audi A3, 2009_2688",Audi,Audi A3,180 000 Kč
4,"Audi A3, 2009_2765",Audi,Audi A3,200 000 Kč


In [859]:
df_auto_cena.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2939 entries, 0 to 2938
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2939 non-null   object
 1   znacka  2939 non-null   object
 2   model   2939 non-null   object
 3   cena    2939 non-null   object
dtypes: object(4)
memory usage: 92.0+ KB


#### Úprava informací

In [860]:
df_info = df_auta[['auto','info']]

In [861]:
df_info_beznan = df_info.dropna(subset=['info'],how='all').reset_index(drop=True)

In [862]:
df_info_beznan

,auto,info
0,"Škoda Karoq, 2021_0",Rok uvedení do provozu 2021\nTachometr 38 188 ...
1,"Škoda Karoq, 2021_0",Karoserie SUV\nPalivo Diesel\nMíst k sezení 5 ...
2,"Škoda Octavia, 2021_1",Rok uvedení do provozu 2021\nTachometr 19 997 ...
3,"Škoda Octavia, 2021_1",Karoserie Hatchback\nPalivo Benzín\nMíst k sez...
4,"Škoda Octavia, 2021_2",Rok uvedení do provozu 2021\nTachometr 21 467 ...
...,...,...
6083,"BMW 318 i, 2019_2936",Karoserie Combi\nPalivo Benzín\nMíst k sezení ...
6084,"Citroen C-Crosser, 2008_2937",Rok uvedení do provozu 2008\nTachometr 228 500...
6085,"Citroen C-Crosser, 2008_2937",Karoserie SUV\nPalivo Diesel\nMíst k sezení 7 ...
6086,"Nissan Juke, 2020_2938",Rok uvedení do provozu 2020\nTachometr 30 886 ...


In [863]:
df_info_beznan['cumcount'] = df_info_beznan.groupby(['auto']).cumcount()+1

In [864]:
info_pivot = df_info_beznan.pivot(index='auto',columns='cumcount',values='info').reset_index()

In [865]:
info_pivot.head()

cumcount,auto,1,2,3,4,5
0,"Alfa Romeo 159, 2007_371",Rok uvedení do provozu 2007\nTachometr 235 177...,Karoserie Combi\nPalivo Diesel\nMíst k sezení ...,NaN,NaN,NaN
1,"Alfa Romeo 159, 2008_1894",Rok uvedení do provozu 2008\nTachometr 196 092...,Karoserie Combi\nPalivo Diesel\nMíst k sezení ...,NaN,NaN,NaN
2,"Alfa Romeo Giulietta, 2018_2882",Rok uvedení do provozu 2018\nTachometr 84 788 ...,Karoserie Hatchback\nPalivo Diesel\nMíst k sez...,NaN,NaN,NaN
3,"Audi A3, 2009_2688",Rok uvedení do provozu 2009\nTachometr 199 896...,Karoserie Hatchback\nPalivo Benzín\nMíst k sez...,NaN,NaN,NaN
4,"Audi A3, 2009_2765",Rok uvedení do provozu 2009\nTachometr 122 236...,Karoserie Hatchback\nPalivo Benzín\nMíst k sez...,NaN,NaN,NaN


In [866]:
info_cast1 = info_pivot[1].str.split(pat="\n", expand=True)

In [867]:
info_cast1 = info_cast1.rename(columns = {0:'rok', 1:'tachometr',2:'motor',3:'prevodovka',4:'barva',5:'vybava_stk',6:'stk',7:'x'})

In [868]:
info_cast1 = info_cast1[['rok','tachometr','motor','prevodovka','barva']]
info_cast1.head()

,rok,tachometr,motor,prevodovka,barva
0,Rok uvedení do provozu 2007,Tachometr 235 177 km,"Motor 1.9 JTD, 110kW",Převodovka 6 stupňů,Barva stříbrná-metalíza
1,Rok uvedení do provozu 2008,Tachometr 196 092 km,"Motor 1.9 JTD, 110kW",Převodovka Automat / 5 stupňů,Barva šedá-metalíza
2,Rok uvedení do provozu 2018,Tachometr 84 788 km,"Motor 1.6 JTDM, 88kW",Převodovka 6 stupňů,Barva šedá-metalíza
3,Rok uvedení do provozu 2009,Tachometr 199 896 km,"Motor 1.8 TFSI, 118kW",Převodovka 6 stupňů,Barva šedá-metalíza
4,Rok uvedení do provozu 2009,Tachometr 122 236 km,"Motor 1.6, 75kW",Převodovka Automat / 7 stupňů,Barva bílá


In [869]:
info_cast2 = info_pivot[2].str.split(pat="\n", expand=True)

In [870]:
info_cast2 = info_cast2.rename(columns = {0:'karoserie', 1:'palivo',2:'mist_sezeni',3:'emisni_norma',4:'emise_servis',5:'servisni_knizka'})

In [871]:
info_cast2 = info_cast2[['karoserie','palivo']]
info_cast2.head()

,karoserie,palivo
0,Karoserie Combi,Palivo Diesel
1,Karoserie Combi,Palivo Diesel
2,Karoserie Hatchback,Palivo Diesel
3,Karoserie Hatchback,Palivo Benzín
4,Karoserie Hatchback,Palivo Benzín


In [872]:
info_pivot[['rok','tachometr','motor','prevodovka','barva']] = info_cast1

In [873]:
info_pivot[['karoserie','palivo']] = info_cast2

In [874]:
info_pivot = info_pivot[['auto','rok','tachometr','motor','prevodovka','barva','karoserie','palivo']]
info_pivot.head()

cumcount,auto,rok,tachometr,motor,prevodovka,barva,karoserie,palivo
0,"Alfa Romeo 159, 2007_371",Rok uvedení do provozu 2007,Tachometr 235 177 km,"Motor 1.9 JTD, 110kW",Převodovka 6 stupňů,Barva stříbrná-metalíza,Karoserie Combi,Palivo Diesel
1,"Alfa Romeo 159, 2008_1894",Rok uvedení do provozu 2008,Tachometr 196 092 km,"Motor 1.9 JTD, 110kW",Převodovka Automat / 5 stupňů,Barva šedá-metalíza,Karoserie Combi,Palivo Diesel
2,"Alfa Romeo Giulietta, 2018_2882",Rok uvedení do provozu 2018,Tachometr 84 788 km,"Motor 1.6 JTDM, 88kW",Převodovka 6 stupňů,Barva šedá-metalíza,Karoserie Hatchback,Palivo Diesel
3,"Audi A3, 2009_2688",Rok uvedení do provozu 2009,Tachometr 199 896 km,"Motor 1.8 TFSI, 118kW",Převodovka 6 stupňů,Barva šedá-metalíza,Karoserie Hatchback,Palivo Benzín
4,"Audi A3, 2009_2765",Rok uvedení do provozu 2009,Tachometr 122 236 km,"Motor 1.6, 75kW",Převodovka Automat / 7 stupňů,Barva bílá,Karoserie Hatchback,Palivo Benzín


In [875]:
info_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2936 entries, 0 to 2935
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   auto        2936 non-null   object
 1   rok         2936 non-null   object
 2   tachometr   2936 non-null   object
 3   motor       2936 non-null   object
 4   prevodovka  2936 non-null   object
 5   barva       2933 non-null   object
 6   karoserie   2936 non-null   object
 7   palivo      2936 non-null   object
dtypes: object(8)
memory usage: 183.6+ KB


#### Join dataframů s cenami a informacemi

In [876]:
auta_merged = df_auto_cena.merge(info_pivot, how='left', left_on='id', right_on='auto')

#### Inspekce nabídek, které neobsahovali informace (NaN hodnoty)

In [877]:
auta_merged[auta_merged.isna().any(axis=1)]

cumcount,id,znacka,model,cena,auto,rok,tachometr,motor,prevodovka,barva,karoserie,palivo
170,"BMW 520d, 2013_2420",BMW,5,330 000 Kč,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,"BMW X5, 2006_94",BMW,X5,240 000 Kč,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
805,"Kia Optima, 2016_2205",Kia,Kia Optima,468 000 Kč,"Kia Optima, 2016_2205",Rok uvedení do provozu 2016,Tachometr 75 736 km,"Motor 2.0 T-GDI GT, 180kW",Převodovka Automat / 6 stupňů,None,Karoserie Combi,Palivo Benzín
1209,"Nissan Qashqai, 2014_1716",Nissan,Qashqai,50 000 Kč,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1483,"Seat Toledo, 2014_1909",Seat,Seat Toledo,170 000 Kč,"Seat Toledo, 2014_1909",Rok uvedení do provozu 2014,Tachometr 145 181 km,"Motor 1.2 TSI, 63kW",Převodovka 5 stupňů,None,Karoserie Sedan,Palivo Benzín
2652,"Škoda Rapid, 2015_1677",Škoda,Škoda Rapid,163 000 Kč,"Škoda Rapid, 2015_1677",Rok uvedení do provozu 2015,Tachometr 158 909 km,"Motor 1.2 TSI, 63kW",Převodovka 5 stupňů,None,Karoserie Hatchback,Palivo Benzín


#### Verifikace, proč chybí data - link je neplatný (asi byla mezitím nabídka stažena), postup je tedy dosud správný

In [878]:
df[df['nadpis'] == 'BMW 520d, 2013_2420'].iloc[0]['url']

'https://www.aaaauto.cz/cz/bmw-5/car.html?id=503616588#&promo=r'

#### Nahrazení chybějících hodnot (barva) nejčastější hodnotou
(ideální by bylo rozkliknout jednotlivé odkazy a barvu nahradit u každého auta zvlášť, v případě mnoha položek by to bylo ale pracné)

In [879]:
modus_barva = auta_merged['barva'].value_counts().idxmax()
modus_barva

'Barva šedá-metalíza'

In [880]:
auta_merged["barva"] = auta_merged["barva"].fillna(modus_barva)

#### Smazání chybějících hodnot, kde chybí všechny informace

In [881]:
auta_merged = auta_merged.dropna().reset_index(drop=True)

In [882]:
auta_merged = auta_merged.drop(['auto'], axis=1)

#### Data už neobsahují žádné chybějící ani duplicitní hodnoty

In [883]:
auta_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2936 entries, 0 to 2935
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          2936 non-null   object
 1   znacka      2936 non-null   object
 2   model       2936 non-null   object
 3   cena        2936 non-null   object
 4   rok         2936 non-null   object
 5   tachometr   2936 non-null   object
 6   motor       2936 non-null   object
 7   prevodovka  2936 non-null   object
 8   barva       2936 non-null   object
 9   karoserie   2936 non-null   object
 10  palivo      2936 non-null   object
dtypes: object(11)
memory usage: 252.4+ KB


#### Smazání zbytečného textu z numerických hodnot a převod na správné datové typy

In [884]:
auta_merged['cena'] = auta_merged['cena'].str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
auta_merged['tachometr'] = auta_merged['tachometr'].str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

In [885]:
auta_merged['rok'] = auta_merged['rok'].str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

In [886]:
auta_merged['rok'] = pd.to_datetime(auta_merged['rok'],format = "%Y").dt.year

#### Všechny proměnné mají nyní správný datový typ

In [887]:
auta_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2936 entries, 0 to 2935
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          2936 non-null   object
 1   znacka      2936 non-null   object
 2   model       2936 non-null   object
 3   cena        2936 non-null   int32 
 4   rok         2936 non-null   int64 
 5   tachometr   2936 non-null   int32 
 6   motor       2936 non-null   object
 7   prevodovka  2936 non-null   object
 8   barva       2936 non-null   object
 9   karoserie   2936 non-null   object
 10  palivo      2936 non-null   object
dtypes: int32(2), int64(1), object(8)
memory usage: 229.5+ KB


In [888]:
auta_merged.head()

cumcount,id,znacka,model,cena,rok,tachometr,motor,prevodovka,barva,karoserie,palivo
0,"Alfa Romeo 159, 2007_371",Alfa Romeo,Alfa Romeo 159,74000,2007,235177,"Motor 1.9 JTD, 110kW",Převodovka 6 stupňů,Barva stříbrná-metalíza,Karoserie Combi,Palivo Diesel
1,"Alfa Romeo 159, 2008_1894",Alfa Romeo,Alfa Romeo 159,106000,2008,196092,"Motor 1.9 JTD, 110kW",Převodovka Automat / 5 stupňů,Barva šedá-metalíza,Karoserie Combi,Palivo Diesel
2,"Alfa Romeo Giulietta, 2018_2882",Alfa Romeo,Alfa Romeo Giulietta,311000,2018,84788,"Motor 1.6 JTDM, 88kW",Převodovka 6 stupňů,Barva šedá-metalíza,Karoserie Hatchback,Palivo Diesel
3,"Audi A3, 2009_2688",Audi,Audi A3,180000,2009,199896,"Motor 1.8 TFSI, 118kW",Převodovka 6 stupňů,Barva šedá-metalíza,Karoserie Hatchback,Palivo Benzín
4,"Audi A3, 2009_2765",Audi,Audi A3,200000,2009,122236,"Motor 1.6, 75kW",Převodovka Automat / 7 stupňů,Barva bílá,Karoserie Hatchback,Palivo Benzín


#### Zkoumání unikátních hodnot a jejich počet v rámci každé proměnné

In [889]:
for sloupec in auta_merged:
    unikatni = np.unique(auta_merged[sloupec])
    pocet_hodnot = len(unikatni)
    if pocet_hodnot < 30:
        print('Pocet hodnot pro promennou {} :{} -- {}'.format(sloupec, pocet_hodnot,unikatni))
    else:
        print('Pocet hodnot pro promennou {} :{}'.format(sloupec, pocet_hodnot))

Pocet hodnot pro promennou id :2936
Pocet hodnot pro promennou znacka :35
Pocet hodnot pro promennou model :231
Pocet hodnot pro promennou cena :423
Pocet hodnot pro promennou rok :18 -- [2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019 2020 2021]
Pocet hodnot pro promennou tachometr :2912
Pocet hodnot pro promennou motor :611
Pocet hodnot pro promennou prevodovka :16 -- ['Celkový výkon 138kW' 'Celkový výkon 185kW' 'Celkový výkon 220kW'
 'Celkový výkon 90kW' 'Celkový výkon 92kW' 'Převodovka'
 'Převodovka 5 stupňů' 'Převodovka 6 stupňů' 'Převodovka 7 stupňů'
 'Převodovka 77 stupňů' 'Převodovka Automat'
 'Převodovka Automat / 5 stupňů' 'Převodovka Automat / 6 stupňů'
 'Převodovka Automat / 7 stupňů' 'Převodovka Automat / 8 stupňů'
 'Převodovka Automat / 9 stupňů']
Pocet hodnot pro promennou barva :37
Pocet hodnot pro promennou karoserie :15 -- ['Karoserie' 'Karoserie Bus' 'Karoserie Cabrio'
 'Karoserie Chladící/mrazící' 'Karoserie Combi' 'Karoserie Coupe'


#### Při zkoumání hodnot jednotlivých proměnných identifikovány u několika nabídek nesprávné hodnoty (kW u převodovky místo motoru apod.)

In [890]:
auta_merged['prevodovka'].value_counts()

Převodovka 6 stupňů              1404
Převodovka Automat / 6 stupňů     538
Převodovka Automat / 7 stupňů     404
Převodovka 5 stupňů               399
Převodovka Automat / 8 stupňů     141
                                 ... 
Celkový výkon 92kW                  1
Celkový výkon 220kW                 1
Celkový výkon 90kW                  1
Převodovka 77 stupňů                1
Převodovka                          1
Name: prevodovka, Length: 16, dtype: int64

In [891]:
seznam_spatnych_prevodovek = ['Celkový výkon 185kW', 'Celkový výkon 138kW', 'Celkový výkon 92kW' , 'Celkový výkon 220kW', 'Celkový výkon 90kW']

#### Nahrazení špatných hodnot u proměnných (převodovka, barva a motor) nejčastější hodnotou

In [892]:
auta_merged[auta_merged['prevodovka'].isin(seznam_spatnych_prevodovek)].loc[:, 'barva']

179     Výkon spal. motoru 135kW
514     Výkon spal. motoru 103kW
715      Výkon spal. motoru 91kW
956     Výkon spal. motoru 183kW
1537     Výkon spal. motoru 72kW
Name: barva, dtype: object

In [893]:
modus_prevodovka = auta_merged['prevodovka'].value_counts().idxmax()
modus_prevodovka

'Převodovka 6 stupňů'

In [894]:
auta_merged.loc[auta_merged['prevodovka'].isin(seznam_spatnych_prevodovek),'prevodovka'] = auta_merged.loc[auta_merged['prevodovka'].isin(seznam_spatnych_prevodovek),'prevodovka'].apply(lambda s: re.sub(r'^.*$', modus_prevodovka, s))

In [895]:
auta_merged.loc[(auta_merged['barva'].str.contains('STK') == 1) | (auta_merged['barva'].str.contains('kW') == 1), 'barva'] = auta_merged.loc[(auta_merged['barva'].str.contains('STK') == 1) | (auta_merged['barva'].str.contains('kW') == 1), 'barva'].apply(lambda s: re.sub(r'^.*$', modus_barva, s))

In [896]:
auta_merged.loc[(auta_merged['motor'].str.contains('kW') == 0), 'motor'] = auta_clean.loc[(auta_clean['motor'].str.contains('kW') == 0), 'barva']

#### Z převodovky vytvoření pouze 2 kategorií = automat a manuál

In [897]:
auta_merged.loc[auta_merged['prevodovka'].str.contains('Převodovka'), 'prevodovka'] = auta_merged.loc[auta_merged['prevodovka'].str.contains('Převodovka'), 'prevodovka'].apply(lambda x: 'Automat' if 'Automat' in x else 'Manuál')

In [898]:
auta_merged['prevodovka'].value_counts()

Manuál     1813
Automat    1123
Name: prevodovka, dtype: int64

#### Z paliva vytvoření pouze následujících kategorií: diesel, benzín, LPG, Hybrid, CNG

In [899]:
auta_merged['palivo'].value_counts()

Palivo Diesel           2015
Palivo Benzín            862
Palivo LPG                38
Palivo Hybrid (FHEV)      10
Palivo CNG                 4
Palivo Benzín (MHEV)       3
Palivo Diesel (MHEV)       2
Palivo Hybrid (MHEV)       1
Palivo Hybrid (PHEV)       1
Name: palivo, dtype: int64

In [900]:
auta_merged['palivo'] = np.select([auta_merged['palivo'].str.contains('Diesel', case=False), auta_merged['palivo'].str.contains('Benzín', case=False), auta_merged['palivo'].str.contains('LPG', case=False),
          auta_merged['palivo'].str.contains('CNG', case=False), auta_merged['palivo'].str.contains('Hybrid', case=False)],
          ['Diesel','Benzín','LPG','CNG','Hybrid'])

In [901]:
auta_merged['palivo'].value_counts()

Diesel    2017
Benzín     865
LPG         38
Hybrid      12
CNG          4
Name: palivo, dtype: int64

#### Smazání zbytečného textu ze stringů

In [902]:
auta_merged['barva'] = auta_merged['barva'].replace({'Barva ': ''}, regex=True)

In [903]:
auta_merged['karoserie'] = auta_merged['karoserie'].replace({'Karoserie ': ''}, regex=True)

In [904]:
auta_merged['motor'] = auta_merged['motor'].replace({'Motor ': ''}, regex=True)

#### Extrakce z proměnné motor -> proměnná výkon (kW)

In [927]:
pd.set_option('display.max_rows', 10)

In [928]:
regex = (r"(\d*[kW].*$)")

In [929]:
auta_merged['motor'] = auta_merged['motor'].str.extract(regex)

In [930]:
regex2 = (r"(\d{2,3})")

In [931]:
auta_merged['motor'] = auta_merged['motor'].str.extract(regex2)

In [933]:
auta_merged['motor'] = auta_merged['motor'].astype(int)

In [934]:
auta_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2936 entries, 0 to 2935
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          2936 non-null   object
 1   znacka      2936 non-null   object
 2   model       2936 non-null   object
 3   cena        2936 non-null   int32 
 4   rok         2936 non-null   int64 
 5   tachometr   2936 non-null   int32 
 6   motor       2936 non-null   int32 
 7   prevodovka  2936 non-null   object
 8   barva       2936 non-null   object
 9   karoserie   2936 non-null   object
 10  palivo      2936 non-null   object
dtypes: int32(3), int64(1), object(7)
memory usage: 218.0+ KB


#### Přejmenování sloupců (jednotky)

In [938]:
auta_merged = auta_merged.rename(columns = {'cena':'cena_kc', 'tachometr':'tachometr_km','motor':'vykon_kw'})

#### Data jsou čistá!

In [939]:
auta_merged

cumcount,id,znacka,model,cena_kc,rok,tachometr_km,vykon_kw,prevodovka,barva,karoserie,palivo
0,"Alfa Romeo 159, 2007_371",Alfa Romeo,Alfa Romeo 159,74000,2007,235177,110,Manuál,stříbrná-metalíza,Combi,Diesel
1,"Alfa Romeo 159, 2008_1894",Alfa Romeo,Alfa Romeo 159,106000,2008,196092,110,Automat,šedá-metalíza,Combi,Diesel
2,"Alfa Romeo Giulietta, 2018_2882",Alfa Romeo,Alfa Romeo Giulietta,311000,2018,84788,88,Manuál,šedá-metalíza,Hatchback,Diesel
3,"Audi A3, 2009_2688",Audi,Audi A3,180000,2009,199896,118,Manuál,šedá-metalíza,Hatchback,Benzín
4,"Audi A3, 2009_2765",Audi,Audi A3,200000,2009,122236,75,Automat,bílá,Hatchback,Benzín
...,...,...,...,...,...,...,...,...,...,...,...
2931,"Škoda Yeti, 2016_1177",Škoda,Škoda Yeti,328000,2016,92427,92,Manuál,hnědá-metalíza,SUV,Benzín
2932,"Škoda Yeti, 2016_194",Škoda,Škoda Yeti,276000,2016,154613,110,Manuál,bílá,SUV,Benzín
2933,"Škoda Yeti, 2016_967",Škoda,Škoda Yeti,365000,2016,134560,81,Manuál,bílá-metalíza,SUV,Diesel
2934,"Škoda Yeti, 2017_1083",Škoda,Škoda Yeti,330000,2017,147493,81,Manuál,bílá,SUV,Diesel


#### Export čistých dat do csv

In [935]:
auta_merged.to_csv('auta_nova.csv',encoding='utf-8-sig')